In [ ]:
import numpy as np
import os

class WSJ():
    """ Load the WSJ speech dataset
        
        Ensure WSJ_PATH is path to directory containing 
        all data files (.npy) provided on Kaggle.
        
        Example usage:
            loader = WSJ()
            trainX, trainY = loader.train
            assert(trainX.shape[0] == 24590)
            
    """
  
    def __init__(self):
        self.dev_set = None
        self.train_set = None
        self.test_set = None
  
    @property
    def dev(self):
        if self.dev_set is None:
            self.dev_set = load_raw('', 'dev')
        return self.dev_set

    @property
    def train(self):
        if self.train_set is None:
            self.train_set = load_raw('', 'train')
        return self.train_set
  
    @property
    def test(self):
        if self.test_set is None:
            self.test_set = (np.load(os.path.join('', 'test.npy'), encoding='bytes'), None)
        return self.test_set
    
def load_raw(path, name):
    return (
        np.load(os.path.join(path, '{}.npy'.format(name)), encoding='bytes'), 
        np.load(os.path.join(path, '{}_labels.npy'.format(name)), encoding='bytes')
    )


loader = WSJ()
trainX, trainY = loader.train
devX, devY = loader.dev
testX, testY= loader.test
assert(trainX.shape[0] == 24590)

In [33]:
import torch
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import time

In [4]:
# write 
lenutterance = len(trainX) 

lenutterance

24590

In [12]:
with open('trainFile.txt', 'w') as the_file:
    for i in range(lenutterance):
        lenframe = len(trainX[i])
        for j in range(lenframe):
            the_file.write(str(i)+","+str(j)+","+str(trainY[i][j])+"\n")


In [22]:
def splitLine(x):
    listLine = x.split(",")
    return int(listLine[0]),int(listLine[1]),int(listLine[2])

class loader(Dataset): 
    def __init__(self, data, list_file, context_length, test_mode=False):
# Question: What about predict the label of the 1st frame? 
#Recall what you have learned from hw0. 
#Question: Why we need specify test_mode? 

        self.data = data 
        self.list_file = open(list_file).readlines() 
        self.w = context_length//2 
        self.test = test_mode
    
    def __getitem__(self, index): 
        one_line_content = self.list_file[index]
        x, y, label = splitLine(one_line_content) 
        data = self.data[x][y] # Qestion: how to have context data?
        return data, label 
    
    def __len__(self): 
        return len(self.list_file)


In [24]:
train=loader(trainX,'trainFile.txt',2)
train_loader = DataLoader(train, batch_size=10, shuffle=True, num_workers=1)

In [27]:
# SIMPLE MODEL DEFINITION
class Simple_MLP(nn.Module):
    def __init__(self, size_list):
        super(Simple_MLP, self).__init__()
        layers = []
        self.size_list = size_list
        for i in range(len(size_list) - 2):
            layers.append(nn.Linear(size_list[i],size_list[i+1]))
            layers.append(nn.ReLU())
        layers.append(nn.Linear(size_list[-2], size_list[-1]))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        x = x.view(-1, self.size_list[0]) # Flatten the input
        return self.net(x)
    

In [29]:
model = Simple_MLP([48, 20, 10])
criterion = nn.CrossEntropyLoss()
cuda = torch.cuda.is_available()
print(model)

optimizer = optim.Adam(model.parameters())
device = torch.device("cuda" if cuda else "cpu")

Simple_MLP(
  (net): Sequential(
    (0): Linear(in_features=784, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=10, bias=True)
  )
)


In [34]:
def train_epoch(model, train_loader, criterion, optimizer):
    model.train()
    model.to(device)

    running_loss = 0.0
    
    start_time = time.time()
    for batch_idx, (data, target) in enumerate(train_loader):   
        optimizer.zero_grad()   
        data = data.to(device)
        target = target.long().to(device)

        outputs = model(data)
        loss = criterion(outputs, target)
        running_loss += loss.item()

        loss.backward()
        optimizer.step()
    
    end_time = time.time()
    
    running_loss /= len(train_loader)
    print('Training Loss: ', running_loss, 'Time: ',end_time - start_time, 's')
    return running_loss

In [ ]:
train_epoch(model,train_loader,criterion,optimizer)